In [23]:
from google.colab import drive
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from transformers import EarlyStoppingCallback
import torch.nn.functional as F

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
stock_data = '/content/drive/MyDrive/stock_data.csv'
df = pd.read_csv(stock_data)

In [5]:
le = LabelEncoder()
df['Sentiment'] = le.fit_transform(df['Sentiment'])

In [6]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def tokenize_function(sentences):
    return tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

In [8]:
X = df['Text']
y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
train_encodings = tokenize_function(X_train.tolist())
test_encodings = tokenize_function(X_test.tolist())

In [10]:
train_labels = torch.tensor(y_train.tolist())
test_labels = torch.tensor(y_test.tolist())

In [11]:
class FinancialSentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
train_dataset = FinancialSentimentDataset(train_encodings, train_labels)
test_dataset = FinancialSentimentDataset(test_encodings, test_labels)

In [13]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    learning_rate=2e-5
)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [15]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.635900,0.624979
2,0.521900,0.433780
3,0.360900,0.414689
4,0.398000,0.393443
5,0.219200,0.442704
6,0.156900,0.505771
7,0.055900,0.783406


TrainOutput(global_step=1015, training_loss=0.34677284803296543, metrics={'train_runtime': 138.2452, 'train_samples_per_second': 335.057, 'train_steps_per_second': 10.489, 'total_flos': 1133038426584960.0, 'train_loss': 0.34677284803296543, 'epoch': 7.0})

In [16]:
predictions = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

In [17]:
accuracy = accuracy_score(y_test, y_pred)
target_names = ['negative', 'positive']
report = classification_report(y_test, y_pred, target_names=target_names)

In [18]:
print(f'RoBERTa Accuracy: {accuracy}')
print(report)

RoBERTa Accuracy: 0.8352027610008628
              precision    recall  f1-score   support

    negative       0.78      0.76      0.77       421
    positive       0.87      0.88      0.87       738

    accuracy                           0.84      1159
   macro avg       0.82      0.82      0.82      1159
weighted avg       0.83      0.84      0.83      1159



In [24]:
def balanced_binary_focal_loss(logits, labels, gamma=2.0, alpha=0.25):

    probs = F.softmax(logits, dim=1)

    true_probs = probs[range(len(labels)), labels]

    true_probs = torch.clamp(true_probs, min=1e-7, max=1 - 1e-7)

    bce_loss = -torch.log(true_probs)

    focal_loss = (alpha * (1 - true_probs).pow(gamma) * bce_loss).mean()

    return focal_loss

In [25]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = balanced_binary_focal_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [26]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.010400,0.035979
2,0.005400,0.045126
3,0.004000,0.077710
4,0.007200,0.068549


TrainOutput(global_step=580, training_loss=0.007412384535539253, metrics={'train_runtime': 71.4351, 'train_samples_per_second': 648.421, 'train_steps_per_second': 20.298, 'total_flos': 647450529477120.0, 'train_loss': 0.007412384535539253, 'epoch': 4.0})

In [28]:
predictions = trainer.predict(test_dataset)
y_pred = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

In [29]:
accuracy = accuracy_score(y_test, y_pred)
target_names = ['Negative', 'Positive']
report = classification_report(y_test, y_pred, target_names = target_names)

In [30]:
print(f'RoBERTa with Focal Loss Accuracy: {accuracy}')
print(report)

RoBERTa with Focal Loss Accuracy: 0.8360655737704918
              precision    recall  f1-score   support

    Negative       0.78      0.76      0.77       421
    Positive       0.87      0.88      0.87       738

    accuracy                           0.84      1159
   macro avg       0.82      0.82      0.82      1159
weighted avg       0.84      0.84      0.84      1159

